In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.select import Select
from tqdm import tqdm_notebook as tqdm, trange

In [2]:
driver = webdriver.Chrome(os.path.join(os.pardir,'chromedriver'))
flow_url = 'https://d2ul3yiqqp2zg9.cloudfront.net/'
driver.get(flow_url)

In [10]:
def string_populate(name,string,driver):
    string_field = driver.find_element_by_name(name)
    string_field.clear()
    string_field.send_keys(string)

def button_click(name,driver):
    button = WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.NAME,name)))
    button.click()
    
def auto_populate(name_list,string_vals,driver):
    if type(string_vals) is dict:
        for name in name_list:
            string = string_vals[name]
            string_populate(name,string,driver)
            
    elif type(string_vals) is list:
        for key,name in enumerate(name_list):
            string = string_vals[key]
            string_populate(name,string,driver)
    else:
        for name in name_list:
            string = string_vals
            string_populate(name,string,driver)
        
def dropdown_option(dropdown_name,option_name,driver):
    dropdown_button = button_click(dropdown_name,driver)
    option_button = button_click(option_name,driver)

def auto_click(name_list,driver):
    for name in tqdm(name_list):
        button = driver.find_element_by_name(name)
        button.click()

In [ ]:
#Login 
login_names = ['username','password']
login_string = ['originationdesk@aurora.com','Niv_2019']
login_button = ['signInSubmitButton']

logging_in = auto_populate(login_names,login_string,driver)
login_button = auto_click(login_button,driver)

In [12]:
#Additional Info
info_names = ['termsheetdisclaimer','title','funding']
info_string = ['disclaimer','test title','many funding']
addInfo = auto_populate(info_names,info_string,driver)
select_currency = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "el-select")))
select_currency.click()

In [58]:
#yes/no
yesNo_elements = driver.find_elements_by_class_name("el-radio-group")
for yn_element in tqdm(yesNo_elements):    
    test = yn_element.find_elements_by_tag_name('span')
    no = [t for t in test if t.text == 'No'][0]
    yes = [t for t in test if t.text == 'Yes'][0]
    driver.execute_script("arguments[0].click();",no)
    

In [59]:
#termsheet working draft - get from json
string_names = ['existing_notes_isin','series_number','common_code',
                'use_of_proceeds','expenses','calculation_agent','series_number',
                'field-nominal_amount','field-net_proceeds_amount','field-denomination_minimum',
                'field-integral_multiple']

In [60]:
termsheet_string = auto_populate(string_names,'test123',driver)

In [61]:
#dates
date_elements = driver.find_elements_by_css_selector(".el-date-editor.el-input.el-input--prefix.el-input--suffix.el-date-editor--date > input")
for date in tqdm(date_elements):
    actionchains = ActionChains(driver)
    date_doubleclick = actionchains.double_click(date).perform()
    date.clear()
    date.send_keys("30-Jun-2021")


In [62]:
#numbers:easy
number_inputs = driver.find_elements_by_xpath("//input[@type='number']")
for number in tqdm(number_inputs):
    number.clear()
    number.send_keys(65)

In [63]:
#add means it is a name list
add_name = ["add-lead_managers"]
buttons = driver.find_elements_by_class_name("loading")
buttons_y = [b.location['y'] for b in buttons]

In [64]:
#locate relavant field, get height of field then apply the tolerances
num_inputs = 3
for name_list in tqdm(add_name):
    name_list = driver.find_element_by_name(name_list)
    name_list_loc = name_list.location
    name_list_size = name_list.size

    #height tolerances
    height_lower = name_list_loc['y'] - name_list_size['height']//2
    height_upper = name_list_loc['y'] + name_list_size['height']//2

    #finding add button for name_list
    button_index = []
    for key,b_y in enumerate(buttons_y):
        if b_y in range(height_lower,height_upper):
            button_index.append(key)

    assert len(button_index) == 1, 'Only one button should be on same level as field, change the height tolerance or reset'
    add_button = buttons[button_index[0]]

#inputing required textlist
    for i in trange(num_inputs):
        key = 'test {}'.format(i+1)
        name_list.send_keys(key)
        if i +1 != num_inputs:
            add_button.click()

100%|██████████| 3/3 [00:01<00:00,  1.52it/s]

In [65]:
#dates
def dates(driver)
    date_elements = driver.find_elements_by_css_selector(".el-date-editor.el-input.el-input--prefix.el-input--suffix.el-date-editor--date > input")
    for date in tqdm(date_elements):
        actionchains = ActionChains(driver)
        date_doubleclick = actionchains.double_click(date).perform()
        date.clear()
        date.send_keys("30-Jun-2021")

#yes/no
def yesno(driver):
    yesNo_elements = driver.find_elements_by_class_name("el-radio-group")
    for yn_element in tqdm(yesNo_elements):    
        test = yn_element.find_elements_by_tag_name('span')
        no = [t for t in test if t.text == 'No'][0]
        yes = [t for t in test if t.text == 'Yes'][0]
        driver.execute_script("arguments[0].click();",no)

#numbers:easy
def number(driver):
    number_inputs = driver.find_elements_by_xpath("//input[@type='number']")
    for number in tqdm(number_inputs):
        number.clear()
        number.send_keys(65)


#successfully found all selectables and can dclick with this
def selectables(driver):
    selectable_inputs = driver.find_elements_by_xpath("//input[@placeholder='Select']")
    for val,selectable in enumerate(selectable_inputs):
        selected = driver.execute_script("arguments[0].click();",selectable)
    #     down = selectable.send_keys(Keys.DOWN)
        enter = selectable.send_keys(Keys.ENTER)